In [262]:
import jsonlines
import pandas as pd
import json
import numpy as np
from scipy.stats import spearmanr
from scipy.stats import kendalltau

In [273]:
input_path_A = 'scores_M2.jsonl' # 'scores_M1.jsonl' 'scores_M2.jsonl' 'scores_gemini200.jsonl'
input_path_B = 'gbt_scores.jsonl'

In [274]:
def get_scores(input_path):
    df = pd.read_json(path_or_buf=input_path, lines=True)
    results = []

    for index, row in df.iterrows():
        completion = row['completion']

        list = []

        for i in range(4):

            list.append(completion[str(i)]['score'])
    
        results.append({index: list})

    return results
            

In [275]:
results_A = get_scores(input_path_A)
results_B = get_scores(input_path_B)

In [276]:
def get_pairwise_accuracy(results_A, results_B):
    correct = 0
    for i in range(len(results_A)):
        current_scores_A = np.array(results_A[i][i])
        current_scores_B = np.array(results_B[i][i])
        difference = current_scores_A - current_scores_B
        current_pairwire_correct = np.count_nonzero(difference == 0)
        correct += current_pairwire_correct
    accuracy = correct / (len(results_A) * 4)
    return accuracy

In [277]:
get_pairwise_accuracy(results_A, results_B)

0.0375

In [268]:
def get_5_best(results_A, results_B):
    total_5 = 0
    correct_5 = 0
    for i in range(len(results_A)):
        
        current_scores_A = np.array(results_A[i][i])
        current_scores_B = np.array(results_B[i][i])
        index_of_best_A = np.where(current_scores_A == 5)[0]
        index_of_best_B = np.where(current_scores_B == 5)[0]
        
        for j in range(len(index_of_best_A)):
            for k in range(len(index_of_best_B)):
                if index_of_best_A[j] == index_of_best_B[k]:
                    correct_5 += 1

        total_5 += len(index_of_best_A)
    accuracy = correct_5 / total_5
    
    return accuracy

In [270]:
get_5_best(results_A, results_B)

0.571072319201995

In [271]:
def get_exact_match(results_A, results_B):
    exact_match = 0
    for i in range(len(results_A)):
        current_scores_A = np.array(results_A[i][i])
        current_scores_B = np.array(results_B[i][i])
        difference = current_scores_A - current_scores_B
        current_pairwire_correct = np.count_nonzero(difference == 0)
        if current_pairwire_correct == 4:
            exact_match += 1
    accuracy = exact_match / len(results_A)
    return accuracy

In [272]:
get_exact_match(results_A, results_B)

0.03

In [238]:
def spearman_corr(results_A, results_B):
    list_A = []
    list_B = []

    for item in results_A:
        for key, value in item.items():
            list_A.extend(value)
    for item in results_B:
        for key, value in item.items():
            list_B.extend(value)

    correlation, p_value = spearmanr(list_A, list_B)
    
    return correlation

In [240]:
spearman_corr(results_A, results_B)

0.34022014676825735

In [241]:
def kendall_tau(results_A, results_B):
    list_A = []
    list_B = []

    for item in results_A:
        for key, value in item.items():
            list_A.extend(value)
    for item in results_B:
        for key, value in item.items():
            list_B.extend(value)

    tau, p_value = kendalltau(list_A, list_B)
    
    return tau

In [242]:
kendall_tau(results_A, results_B)

0.3022169281141561